In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fs = 55
block_size = 16
nfft = 16
num_sample = 1600
hop_size = 16
freq = 11
phase = 0
t = np.arange(num_sample)/fs
f = np.fft.fftfreq(block_size,1/fs )
cs = np.cos(2*np.pi*20*t+phase) 
noise = np.random.normal(0,1.5,num_sample) 
s = cs+0

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(t,s)
plt.tick_params('both',labelsize=25)
plt.grid()
plt.xlabel('time(s)', fontsize=25)
plt.ylabel('rider', fontsize=25)


In [ ]:
s_reshape = np.array([
    s[hop_size*i:hop_size*i+block_size] for i in range(num_sample) if hop_size*i+block_size<=num_sample
])

In [ ]:
dft = np.fft.fft(s_reshape,axis=1)
dht = np.real(dft)-np.imag(dft)

psd_dft = (1/block_size*np.abs(dft)**2).mean(0)
psd_dht = (1/block_size*np.abs(dht)**2).mean(0)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(psd_dft,label='dft')
plt.plot(psd_dht,label='dht')
plt.tick_params('both',labelsize=25)
plt.grid()
plt.legend(fontsize=25)


In [ ]:
e_final = []
theo_es = []
e_final_percent = []
theo_es_percent = []
for N in np.arange(16,128,1):
    fs = 55
    b=N
    block_size = b
    nfft = b
    num_sample = b*800

    hop_size = b
    phase =0

    
    t = np.arange(num_sample)/fs
    f = np.fft.fftfreq(block_size,1/fs )
    
    f=21.479
    cs = np.cos(2*np.pi*f*t+phase) 
    noise = np.random.normal(0,0,num_sample) 
    s = cs
    
    s_reshape = s.reshape(int((num_sample)/N), N)
    
    dft = np.fft.fft(s_reshape,axis=1)
    dht = np.real(dft)-np.imag(dft)
    psd_dft = (1/block_size*np.abs(dft)**2).mean(0)
    psd_dht = (1/block_size*np.abs(dht)**2).mean(0)
    
    block_dft = (1/block_size*np.abs(dft)**2)
    block_dht = (1/block_size*np.abs(dht)**2)
    block_dft_rm = np.array([block_dft[0:i].mean(0) for i in range(1,len(block_dft))])
    block_dht_rm = np.array([block_dht[0:i].mean(0) for i in range(1,len(block_dht))])
    
    w=2*np.pi*f/fs
    bins = np.array([fs*i/N for i in range(N)])
    idx_left = np.where((f-bins)>0)[0][-1]
    idx_right = idx_left+1
    if np.abs(idx_left*fs/N-f) < np.abs(idx_right*fs/N-f):
        k = idx_left
    else:
        k = idx_right
#     print(np.argmax(block_dft_rm[-1][0:int(N/2)]))
#     print(np.max(block_dft_rm[-1][0:int(N/2)]))
#     print(np.abs(idx_left-f))
#     print(np.abs(idx_right-f))
#     print(idx_left*fs/N,idx_right*fs/N,f,idx_left,idx_right)
    theo_e = -np.true_divide(1-np.cos(N*w),2*np.cos(2*np.pi*k/N)-2*np.cos(w))*np.sin(2*np.pi*k/N)
    
    theo_e = 1/N * theo_e
    e_k = block_dht_rm[:,k]-block_dft_rm[:,k]
    theo_es.append(theo_e/N)
    e_final.append(e_k[-1]/N)
    
    theo_es_percent.append((theo_e/N)/(psd_dft[k]/N))
    e_final_percent.append((e_k[-1]/N)/(psd_dft[k]/N))

    error = (block_dht_rm[:,6]-block_dft_rm[:,6])
#     plt.figure(figsize=(8,5))
#     plt.plot(error/N,label='Blocked Average Error',c='r')
#     #plt.plot(error[-1]*np.ones_like(error)/N,label='Asymptote Blocked Average Error',c='k',linestyle='--')
#     plt.grid()
#     plt.xlabel('$l$'+' (block idx)',fontsize=15)
#     #plt.ylabel('$\\frac{\\overline{E_k}}{|X_k|^2}$'+'(%)',fontsize=15)
#     plt.ylabel('Blocked Averaged Error',fontsize=15)
#     #plt.title('Running Average of DHT/DFT Squared Magnitude at k=6',fontsize=15)
#     plt.tick_params('both',labelsize=15)
#     plt.ylim([-1,1])
#     plt.xlim([0,200])
    
#     plt.savefig('dht_avg.jpg')
#     plt.figure(figsize=(8,5))
#     plt.plot(block_dht[:,6]/block_size,label='dht',c='b')
#     plt.plot(block_dft[:,6]/block_size,label='dft',c='k')

#     plt.grid()
#     plt.xlabel('$l$'+' (block idx)',fontsize=15)
#     plt.ylabel('Normalized Squared Magnitude',fontsize=15)
#     #plt.title('DHT Squared Magnitude at k=6',fontsize=15)
#     plt.tick_params('both',labelsize=15)
#     plt.ylim([-0.25,0.6])
#     plt.xlim([0,200])
#     plt.legend(fontsize=15)
#     plt.savefig('dht.jpg')


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(np.arange(16,128,1),100*(np.array(e_final_percent)),c='r',label='simulation')
plt.plot(np.arange(16,128,1),100*(np.array(theo_es_percent)),c='navy',label='analysis')
plt.grid()
plt.xlabel('$N$',fontsize=15)
plt.ylabel('$\\frac{\\overline{E_k}(\infty)}{|X_k|^2}$'+'(%)',fontsize=15)
plt.tick_params('both',labelsize=15,)
plt.ylim([-20,20])
plt.legend(fontsize=15)
plt.savefig('ss_percent.jpg')

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(np.arange(16,128,1),(np.array(e_final)),c='r',label='simulation')
plt.plot(np.arange(16,128,1),(np.array(theo_es)),c='navy',label='analysis')
plt.grid()
plt.xlabel('$N$',fontsize=15)
plt.ylabel('$\\overline{E_k}(\infty)$',fontsize=15)
plt.tick_params('both',labelsize=15,)
plt.ylim([-0.025,0.025])
plt.legend(fontsize=15)
plt.savefig('ss.jpg')

In [ ]:
e_final[-1]

In [ ]:
e_final[-1]

In [ ]:
e_final[-1]

In [ ]:
100*np.array(e_final)

In [ ]:
e_final[-1]

In [ ]:
e_final[-1]

In [ ]:
100*np.array(e_final)

In [ ]:
imgs = []
import imageio
for i in range(50,51):
    N=16
    width=0.3
    fig, ax = plt.subplots(figsize=(10,5))
    ax.bar(np.arange(int(N/2))-width/2,block_dft[i,:][0:int(N/2)],width,label='dft')
    ax.bar(np.arange(int(N/2))+width/2,block_dht[i,:][0:int(N/2)],width,label='dht')
    ax.grid()
    
    ax.set_ylabel('square magnitude',fontsize=15)
    ax.set_xlabel('frequency idx', fontsize=15)
    ax.set_title('DHT vs. DFT per Block', fontsize=15)
    ax.tick_params('both',labelsize=25)

    ax.legend(loc='upper left',fontsize=25)
    
    ax.set_ylim([0,8])
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    imgs.append(image)
    
kwargs_write = {'fps':1.0, 'quantizer':'nq'}
imageio.mimsave('./powers.gif', imgs, fps=2)


In [ ]:
# dft = np.fft.fft(s_reshape,axis=1)
# dht = np.real(dft)-np.imag(dft)
# psd_dft = (1/block_size*np.abs(dft)**2).mean(0)
# psd_dht = (1/block_size*np.abs(dht)**2).mean(0)

In [ ]:
plt.figure(figsize=(20,10))
width = 0.3
plt.bar(np.arange(8)-width/2,psd_dft[0:8],width,label='dft')
plt.bar(np.arange(8)+width/2,psd_dht[0:8],width,label='dht')
plt.tick_params('both',labelsize=25)
plt.grid()
plt.xlabel('frequency idx',fontsize=25)
plt.ylabel('Square Magnitude',fontsize=25)
plt.legend(fontsize=25)

In [ ]:
psd_dft-psd_dht

In [ ]:
dft = np.fft.fft(s_reshape,axis=1)
dht = np.real(dft)-np.imag(dft)

psd_dft = (1/block_size*np.abs(dft)**2).mean(0)
psd_dht = (1/block_size*np.abs(dht)**2).mean(0)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(psd_dft,label='dft')
plt.plot(psd_dht,label='dht')
plt.tick_params('both',labelsize=25)
plt.grid()
plt.legend(fontsize=25)

In [ ]:
f = np.linspace(-27.5,27.5,1001)
w = 2*np.pi*f/fs
N = 16

plt.figure(figsize=(20,10))

for N in [16]:
    k=6
    A = w + 2*np.pi*k/N
    B = w - 2*np.pi*k/N
    term1 = 0.25*(np.sin(N/2*A)/np.sin(A/2))**2 * np.sin((N-1)*A)
    term2 = -0.25*(np.sin(N/2*B)/np.sin(B/2))**2 * np.sin((N-1)*B)
    term3 = -np.true_divide(1-np.cos(N*w),2*np.cos(2*np.pi*k/N)-2*np.cos(w))*np.sin(2*np.pi*k/N)
    result = (term1+term2+term3)/N
    
    real = np.sin(N*A/2)/2/np.sin(A/2) * np.cos((N-1)*A/2) + np.sin(N*B/2)/2/np.sin(B/2) * np.cos((N-1)*B/2)
    imag = np.sin(N*A/2)/2/np.sin(A/2) * np.sin((N-1)*A/2) + np.sin(N*B/2)/2/np.sin(B/2) * np.sin((N-1)*B/2)
    x_k_square = 1/N * (real**2+imag**2)
    
    final = x_k_square + result 
    plt.plot(f[int(len(f)/2):],result[int(len(f)/2):],label='$E_k(w_0)$')
    plt.plot(f[int(len(f)/2):],x_k_square[int(len(f)/2):],label='$|X_k(w_0)|^2$')
    plt.plot(f[int(len(f)/2):],final[int(len(f)/2):],label='$|H_k(w_0)|^2$')
    
    
plt.grid()
plt.tick_params('both',labelsize=25)
plt.xlabel('f(Hz)',fontsize=25)
plt.legend(fontsize=25)
plt.ylabel('Square Magnitude')


In [ ]:
result.max()

In [ ]:
dft = np.fft.fft(s_reshape,axis=1)
dht = np.real(dft)-np.imag(dft)

psd_dft = (1/block_size*np.abs(dft)**2).mean(0)
psd_dht = (1/block_size*np.abs(dht)**2).mean(0)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(psd_dft,label='dft')
plt.plot(psd_dht,label='dht')
plt.tick_params('both',labelsize=25)
plt.grid()
plt.legend(fontsize=25)

In [ ]:
f = np.linspace(0,55,1000)
fs=55
omega0 = 2*np.pi*f/fs
N=16
k=6

def error(f):
    omega0 = 2*np.pi*f/fs
    return (np.cos(N*omega0)-1)/(2*np.cos(2*np.pi*k/N)-2*np.cos(omega0))*np.sin(2*np.pi*k/N)/N

c_w0 = (np.cos(N*omega0)-1)/(2*np.cos(2*np.pi*k/N)-2*np.cos(omega0))*np.sin(2*np.pi*k/N)/N


In [ ]:
plt.figure(figsize=(20,10))
plt.plot(f[0:int(len(f)/2)],c_w0[0:int(len(f)/2)])
plt.grid()
plt.tick_params('both',labelsize=25)
plt.xlabel('$f_o$',fontsize=25)
plt.ylabel('$C(f_o)$',fontsize=25)
plt.title('$C(f_o)$',fontsize=25)

bin_freq = np.array([n/N * fs for n in range(int(N/2))])
special = np.array([])
plt.scatter(bin_freq[6],np.zeros_like(bin_freq)[6],color='red',marker='*',s=600)


In [ ]:
fs = 55
f = np.linspace(0,fs,1000)
pi = np.pi
sin = np.sin
cos = np.cos

omega0 = 2*pi*f/fs
k=6
A = omega0 + 2*pi*k/N
B = omega0 - 2*pi*k/N

U = 32
phis = [i*pi/U for i in range(U+1)]

imgs = []
for idx, phi in enumerate(phis):
    left_pulse = 1/(4*N) * ((sin(N*A/2)/sin(A/2))**2) * sin((N-1)*A + 2*phi)
    right_pulse = 1/(4*N) * ((sin(N*B/2)/sin(B/2))**2) * sin((N-1)*B + 2*phi)
    overall = 1/N * (1-cos(N*omega0))/(2*cos(2*pi*k/N)-2*cos(omega0)) * sin(2*pi*k/N)
    error = left_pulse - right_pulse - overall

    sequence = np.array([cos(omega*np.arange(N) + phi) for omega in omega0])
    fft_sequence = np.fft.fft(sequence, axis=1)
    square_mag = 1/N*np.abs(fft_sequence)**2
    
    if idx == 0:
        ERROR = error 
        SQUARE_MAG = square_mag
        
    fig, ax = plt.subplots(figsize=(20,10))
    
    ax.plot(f[0:int(len(f)/2)],ERROR[0:int(len(f)/2)],c='r',linestyle='--')
    ax.plot(f[0:int(len(f)/2)], SQUARE_MAG[:,6][0:int(len(f)/2)],c='k',linestyle='--')
    ax.plot(f[0:int(len(f)/2)], ERROR[0:int(len(f)/2)]+SQUARE_MAG[:,6][0:int(len(f)/2)],linestyle='--',c='b')

    ax.plot(f[0:int(len(f)/2)],error[0:int(len(f)/2)],label='$E_k(\omega_o,\phi)$',c='r',linewidth=4)
    ax.plot(f[0:int(len(f)/2)], square_mag[:,6][0:int(len(f)/2)],label='$|X_k(\omega_o,\phi)|^2$',c='k')
    ax.plot(f[0:int(len(f)/2)], error[0:int(len(f)/2)]+square_mag[:,6][0:int(len(f)/2)],label='$|H_k(\omega_o,\phi)|^2$',c='b')
    ax.grid()
    ax.legend(fontsize=25)
    ax.tick_params('both',labelsize=25)
    ax.set_xlabel('$f_o=\omega_of_s/2\pi$',fontsize=25)
    ax.set_ylabel('Squared Magnitude',fontsize=25)
    ax.set_ylim([-5,10])
    ax.set_title('$\phi$='+str(idx)+'$\pi/32$',fontsize=25)
    
    ax.scatter([20.625],[0],marker='*',s=300,c='darkviolet')
    fig.canvas.draw()       # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image  = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    imgs.append(image)
    
imageio.mimsave('./powers.gif', imgs, fps=4)

In [ ]:
def error(f,fs,N):
    
    def error_bin(f,fs,N,k):
        omega0 = 2*np.pi*f/fs
        c_w0 = (np.cos(N*omega0)-1)/(2*np.cos(2*np.pi*k/N)-2*np.cos(omega0))*np.sin(2*np.pi*k/N)/N
        return c_w0
    u= [error_bin(f,55,N,k)for k in range(N)]
    return np.sum(u)




In [ ]:
Es = []
for N in [16,32,64,128,256,512,1024,2048,4096,8192]:
    E = error(21.479,55,N)
    Es.append(E)

In [ ]:
plt.plot(Es)